In [1]:
# import libraries

'''
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer
'''

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

# These are all the imports needed for the assignment
%matplotlib inline

# Import nltk package (Natural Language Toolkit)
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings('ignore')

# scikit-learn imports
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, precision_recall_fscore_support

In [2]:
import pyspark

In [3]:
import dask.dataframe as dd

In [4]:
# Download the NLTK English tokenizer and the stopwords of all languages
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sukan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sukan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

client = boto3.client(
    's3',
    aws_access_key_id = 'sukiK@2002',
    aws_secret_access_key = 'sukik',
    region_name = 'ap-west-1'
)

In [5]:
files = ['amazon_reviews_us_Electronics_v1_00.tsv', \
         'amazon_reviews_us_Gift_Card_v1_00.tsv', \
         'amazon_reviews_us_Major_Appliances_v1_00.tsv', \
         'amazon_reviews_us_Office_Products_v1_00.tsv', \
         'amazon_reviews_us_Shoes_v1_00.tsv', \
         'amazon_reviews_us_Toys_v1_00.tsv', \
         'amazon_reviews_us_Watches_v1_00.tsv']

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [7]:
columns = ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', \
           'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date']
elec_df = pd.read_csv('fars_data/' + files[0], names = columns, sep = '\t').iloc[1:,:]
elec_df = elec_df.sample(n = 200000)

columns = ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date']
len(columns)

one_file = pd.read_csv('fars_data/' + files[0], names = columns, sep = '\t').iloc[1:,:]
one_file.head()

one_file.shape[0]+1

In [8]:
one_file = elec_df

## Data Sampling

In [9]:
def df_sampling(df):
    # Since we know that there are more unverified than verified --> we sample based on that
    
    # Since there are no data values in 'verified_purchase' columns that deviate from 'Y' or 'N' we proceed
    verified_count_df = df[df['verified_purchase'] == 'Y']
    unverified_count_df = df[df['verified_purchase'] == 'N']
    
    print("Number of verified purchases:", len(verified_count_df))
    print("Number of unverified purchases:", len(unverified_count_df))
    
    sample_len = len(unverified_count_df)
    
    verified_sample_df = verified_count_df.sample(n = sample_len)
    unified_df = pd.concat([unverified_count_df, verified_sample_df])
    
    print("Number of verified purchases (balanced dataset):", len(unified_df[unified_df['verified_purchase'] == 'Y']))
    print("Number of unverified purchases (balanced dataset):", len(unified_df[unified_df['verified_purchase'] == 'N']))
    
    return unified_df

#test.rename(columns=test.iloc[0])
training_data = pd.DataFrame() #initialize an empty dataframe
testing_data = pd.DataFrame()

one_file = pd.read_csv(data_location + files[1], names = columns, sep = '\t').iloc[1:,:]

In [10]:
one_file.shape

(200000, 15)

In [11]:
balanced_elec = df_sampling(one_file)
display(balanced_elec.head())

Number of verified purchases: 180710
Number of unverified purchases: 19290
Number of verified purchases (balanced dataset): 19290
Number of unverified purchases (balanced dataset): 19290


marketplace  customer_id       review_id  product_id  product_parent  \
165832           US     44099256   RM8UXQHVQADBP  B00SMVLUNK       950751339   
985185           US     26295942   R3ITTTTJ0BVTB  B003OC6LWM       457087873   
1042480          US     17403950   R37CZLBWSNC0E  B00AI5V3CQ       410477650   
217536           US     27354910  R2SDJSDT7E0S2V  B00OEI9AKU       633226384   
1145096          US      3508533  R1TWUQZ2TV2IAS  B00DXV9KIQ       620216527   

                                                                                                                                                                                                   product_title  \
165832                                                                                                                                  Panasonic RP-BTD10-K Premium Bluetooth Wireless On-Ear Headphones, Black   
985185                                                                                                                                      C&E Mini_Dis-HDMI-CB6 Mini DisplayPort to HDMI Adapter Cable, 6 feet   
1042480                 Cambridge SoundWorks OontZ Angle Ultra Portable Wireless Bluetooth Speaker with Built in Mic up to 12 Hour Playtime works with iPhone iPad tablet Samsung and smart phones - Blue Grille   
217536                                                                            8pcs (CL-BL)- Large (L) Replacement Set Earbuds Eartips for Jaybird Bluebuds X Premium Bluetooth In Ear Earphones / Headphones   
1145096  Original Sony RM-YD065 LCD TV Remote Control for KDL-22BX320, KDL-32BX320, KDL-32BX321, KDL-32EX340, KDL-32BX420, KDL-32BX421, KDL-40BX420, KDL-40BX421 KDL-40BX450 KDL-46BX420 KDL-46BX421 KDL-55BX520   

        product_category  star_rating  helpful_votes  total_votes vine  \
165832       Electronics            4              2            3    Y   
985185       Electronics            3              1            1    N   
1042480      Electronics            5              2            3    N   
217536       Electronics            5              0            0    N   
1145096      Electronics            3              0            1    N   

        verified_purchase  \
165832                  N   
985185                  N   
1042480                 N   
217536                  N   
1145096                 N   

                                                                 review_headline  \
165832   Decent cans that look dorky, feel ok, sound mid-range, and travel well.   
985185                                      Good Product - Shipping took a month   
1042480                  Leave Your Phone, Bring This Into The Bathroom Instead.   
217536     They work great. They definitely removed the irritation of my old ...   
1145096                                                              Three Stars   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

### Type Conversion and Exploration

In [12]:
def convert_to_int(x):
    return int(float(x))

In [13]:
#convert all mixed datatypes --> string objects (unable to convert to int)
one_file['customer_id'] = one_file['customer_id'].apply(int) #1
one_file['product_parent'] = one_file['product_parent'].apply(int) #4
one_file['star_rating'] = one_file['star_rating'].apply(int) #7
one_file['helpful_votes'] = one_file['helpful_votes'].apply(int) #8
one_file['total_votes'] = one_file['total_votes'].apply(int) #9

one_file.isnull().sum().sort_values(ascending=False)

one_file.dropna(inplace=True) #drop all of the missing values
one_file.isnull().sum().sort_values(ascending=False)

In [14]:
#there may be tabs in the review that is what is leading to the excluded cases when converting to df

'''
train_data['verified_purchase'].value_counts()
test_data['verified_purchase'].value_counts()
'''

one_file.head()

marketplace  customer_id       review_id  product_id  product_parent  \
416017          US     12501109   RPT39BBGUU0QP  B000WYVBR0       851902359   
789391          US      7816587  R20BTWW2WZSTE8  B00H8WFKGM       410330719   
165832          US     44099256   RM8UXQHVQADBP  B00SMVLUNK       950751339   
37046           US     51731065  R1EQBX510QPH6V  B0040ITOAA       382901065   
644348          US      7443025  R24BIKYABJCICH  B007JY58FM       732571169   

                                                                                                                                                                                               product_title  \
416017  VideoSecu ML531BE TV Wall Mount for most 22"-55" LED LCD Plasma Flat Screen Monitor up to 88 lb VESA 400x400 with Full Motion Swivel Articulating 20 in Extension Arm, HDMI Cable & Bubble Level WP5   
789391                                                                                   Cheetah Mounts AS1B Single Shelf TV Component Wall Mount 18x16-Inch Shelving Bracket Bundle with 15-Feet HDMI Cable   
165832                                                                                                                              Panasonic RP-BTD10-K Premium Bluetooth Wireless On-Ear Headphones, Black   
37046                                                                                                                                                                   Pyle PHSTD1 Satellite Speaker Stands   
644348                                                                                                                                                New Mitsubishi/Toshiba 4719-001997 DLP Chip 1910-6143W   

       product_category  star_rating  helpful_votes  total_votes vine  \
416017      Electronics            5              0            0    N   
789391      Electronics            5              0            0    N   
165832      Electronics            4              2            3    Y   
37046       Electronics            3              1            1    N   
644348      Electronics            5              0            0    N   

       verified_purchase  \
416017                 Y   
789391                 Y   
165832                 N   
37046                  Y   
644348                 Y   

                                                                review_headline  \
416017                                                               Five Stars   
789391                          Great shelves, with extras(HDMI cable and light   
165832  Decent cans that look dorky, feel ok, sound mid-range, and travel well.   
37046                                                                 Just Okay   
644348                Installation was really quite easy, took maybe 20 minutes   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

#to check te datatypes within each column

print(train_data.applymap(type))
data_types = train_data.applymap(type)

In [15]:
#the number of customers that gave multiple reviews
#may be valueable to investigate
one_file.groupby('customer_id').count()['marketplace'].sort_values(ascending = False).value_counts()

1     175987
2       8943
3       1253
4        292
5        108
6         27
7         15
8         12
10         9
9          7
11         3
12         2
13         2
14         1
16         1
31         1
Name: marketplace, dtype: int64

In [16]:
one_file['star_rating'].value_counts()

5    121807
4     31265
1     21754
3     14757
2     10417
Name: star_rating, dtype: int64

## Data Cleaning

In [17]:
# Import re for text cleaning purposes
import re

import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sukan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sukan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
def df_cleaning(df, col):
    # Drop rows with na values
    df.dropna(inplace = True)
    
    new_col_name = 'new_' + col
    
    df[new_col_name] = df[col].copy() 
    
    # Remove unwanted formatting characters
    format_strs = dict.fromkeys(['<br /><br />', '&#34', 'br', '&quot', '<br />'], ' ')
    
    for key in format_strs:
        df[new_col_name] = df[new_col_name].apply(lambda review: review.replace(key, format_strs[key]))
    # removing quotes produces smthg like this --> 'The product has great ;sound; --> we must remove punctuation

    
    # Case normalization (lower case)
    df[new_col_name] = df[new_col_name].str.lower()
    
    remove_dict = {"0": "", "1": "", "2": "", "3": "", "4": "", "5": "", "6": "", "7": "", "8": "", "9": "",
                   "(": "", ")":""}
    for key, val in remove_dict.items():
        df[new_col_name] = df[new_col_name].apply(
            lambda x: x.replace(key, val))
        
    # Remove stopwords
    stop_lst = stopwords.words('english')
    #stop_lst += (["can't", "i'm" "i'd", "i've", "i'll", "that's", "there's", "they're"])
    # ****Do we not have to take stopwords out BEFORE removing punctuation? Otherwise words with punct like “cant” remains there
    df[new_col_name] = df[new_col_name].apply(lambda text_body: " ".join([word for word in text_body.split() if word not in (stop_lst)]))
    
    # Removing Unicode Chars (punctuation, URL, @)
    df[new_col_name] = df[new_col_name].apply(lambda rev: re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", rev))
    
    # Lemmatization
    word_lemmatizer = WordNetLemmatizer()
    df[new_col_name] = df[new_col_name].apply(lambda txt: " ".join([(word_lemmatizer.lemmatize(word)) for word in txt.split()]))
    
    return df

cleaned = df_cleaning(one_file)

In [19]:
cleaned2 = df_cleaning(one_file, 'review_body')

In [20]:
cleaned2.get(['review_body', 'new_review_body']).head()

review_body  \
416017                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [21]:
cleaned2 = df_cleaning(one_file, 'review_headline')

In [22]:
cleaned2.get(['review_headline', 'new_review_headline']).head()

,review_headline,new_review_headline
416017,Five Stars,five star
789391,"Great shelves, with extras(HDMI cable and light",great shelf extrashdmi cable light
165832,"Decent cans that look dorky, feel ok, sound mid-range, and travel well.",decent can look dorky feel ok sound midrange travel well
37046,Just Okay,okay
644348,"Installation was really quite easy, took maybe 20 minutes",installation really quite easy took maybe minute


In [23]:
cleaned2 = df_cleaning(one_file, 'product_title')

In [24]:
cleaned2.get(['product_title', 'new_product_title']).head()

,product_title,new_product_title
416017,"VideoSecu ML531BE TV Wall Mount for most 22""-55"" LED LCD Plasma Flat Screen Monitor up to 88 lb VESA 400x400 with Full Motion Swivel Articulating 20 in Extension Arm, HDMI Cable & Bubble Level WP5",videosecu mlbe tv wall mount led lcd plasma flat screen monitor lb vesa x full motion swivel articulating extension arm hdmi cable bubble level wp
789391,Cheetah Mounts AS1B Single Shelf TV Component Wall Mount 18x16-Inch Shelving Bracket Bundle with 15-Feet HDMI Cable,cheetah mount asb single shelf tv component wall mount xinch shelving bracket bundle foot hdmi cable
165832,"Panasonic RP-BTD10-K Premium Bluetooth Wireless On-Ear Headphones, Black",panasonic rpbtdk premium bluetooth wireless onear headphone black
37046,Pyle PHSTD1 Satellite Speaker Stands,pyle phstd satellite speaker stand
644348,New Mitsubishi/Toshiba 4719-001997 DLP Chip 1910-6143W,new mitsubishitoshiba dlp chip w


In [25]:
cleaned2.iloc[0].get('review_body')

'works good'

## Vader Sentiment Analysis

In [26]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

def get_sentiment_scores(review):
    """
    create new dataframe with just the proportions for each review
    four columns
    neg_prop, pos_prop, neu_prop, compound_prop and will contain these values
    obtained from the vator sentiment algorithm
    """
    snt = analyser.polarity_scores(review)
    #print(f"{sentence} {str(snt)}")
    return snt

In [27]:
get_sentiment_scores(cleaned2.iloc[0].get('review_body')) #without lemmitization

{'neg': 0.0, 'neu': 0.256, 'pos': 0.744, 'compound': 0.4404}

In [28]:
get_sentiment_scores(cleaned2.iloc[0].get('new_review_body')) #with lemmitization

{'neg': 0.0, 'neu': 0.256, 'pos': 0.744, 'compound': 0.4404}

In [29]:
one_file = cleaned2

In [30]:
one_file['rev_dict'] = one_file['new_review_body'].apply(get_sentiment_scores)

In [31]:
def get_neg(review_dict):
    return review_dict['neg']

def get_neu(review_dict):
    return review_dict['neu']

def get_pos(review_dict):
    return review_dict['pos']

def get_compound(review_dict):
    return review_dict['compound']

def only_compound(x):
    dct = get_sentiment_scores(x)
    return dct['compound']

In [32]:
#get neg prop
one_file['neg_prop'] = one_file['rev_dict'].apply(get_neg)
#get neu prop
one_file['neu_prop'] = one_file['rev_dict'].apply(get_neu)
#get pos prop
one_file['pos_prop'] = one_file['rev_dict'].apply(get_pos)
#get compound prop
one_file['compound_prop'] = one_file['rev_dict'].apply(get_compound)

In [33]:
#save the dataframe as a csv file
one_file.to_csv('electronics_data_cleaned.csv')

### Sentiment Analysis

#df is all data other than rating 3
df = one_file[one_file['star_rating'] != 3]

df['star_rating'].value_counts()

df['positively_rated'] = np.where(df['star_rating'] > 3, 1, 0)

df['positively_rated'].value_counts()

sns.countplot(df['positively_rated'])

##### On all of the data

train_data, test_data = np.split(one_file.sample(frac=1, random_state=1729), [int(0.7 * len(one_file))])
print(train_data.shape, test_data.shape)

X_train = train_data['review_body']
X_train.iloc[0] #to_frame()

Y_train = train_data['star_rating']

X_test = test_data['review_body']
Y_test = test_data['star_rating']

#### TF-IDF - to get predictions of what star rating will be based on text of review

from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer().fit(X_train)

len(vect.get_feature_names()) #unique words

X_train_vect = vect.transform(X_train)

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_vect, Y_train)

predictions = model.predict(vect.transform(X_test))

len(predictions)

###### Confusion Matrix - shows probabilities for whether cat data is predicted correctly

In [34]:
def plot_confusion_matrix(cm, target_names,
                          fname, epoch,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True, target=None):
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools
    plt.style.use('default')

    # # only true if it weren't normalized:
    # accuracy = np.trace(cm) / float(np.sum(cm))
    # misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    cm[np.isnan(cm)] = 0.0

    fig = plt.figure(figsize=(5, 4))
    ax = plt.axes()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    if target == "rule-based":
        plt.title(title + ' for rule-based PF')
    else:
        plt.title(title + ' for MLPF at epoch ' + str(epoch))

    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.2f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlim(-1, len(target_names))
    plt.ylim(-1, len(target_names))
    plt.xlabel('Predicted label')
    # plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.tight_layout()
    plt.savefig(fname + '.png')
    plt.savefig(fname + '.pdf')
    #plt.close(fig)

    return fig, ax

### SVM Notes

- **Bag of Words (BoW)**
        - BoW encodes an input sentence as the frequency of each word in the sentence. 
        - In this approach, all words contribute equally to the feature vectors.
- **Term Frequency - Inverse Document Frequency (TF-IDF)**
        - TF-IDF is a measure of how important each term is to a specific document, as compared to an overall corpus. 
        - TF-IDF encodes each word as its frequency in the document of interest, divided by a measure of how common the word is across all documents (the corpus).
        - Using this approach, each word contributes differently to the feature vectors.
        - The assumption behind using TF-IDF is that words that appear commonly everywhere are not that informative about what is specifically interesting about a document of interest, so it is tuned to representing a document in terms of the words it uses that are different from other documents. 

- To compare those 2 methods, we will first apply them on the same dataset to analyse sentiment (how positive or negative a text is). In order to make the comparison fair, an **SVM (support vector machine)** classifier will be used to classify positive reviews and negative reviews.

- SVM is a simple yet powerful and interpretable linear model. To use it as a classifier, we need to have at least 2 splits of the data: training data and test data. The training data is used to tune the weight parameters in the SVM to learn an optimal way to classify the training data. We can then test this trained SVM classifier on the test data, to see how well it works on data that the classifier has not seen before. 

We will now create a CountVectorizer object to transform the text data into vectors with numerical values.

To do so, we will initialize a CountVectorizer object, and name it as vectorizer.

4 arguments to initialize a CountVectorizer:

* analyzer: 'word'
 Specify to analyze data from word-level.
 
* max_features: 2000
 Set a max number of unique words.
 
* tokenizer: word_tokenize
 Set to tokenize the text data by using the word_tokenizer from NLTK .
 
* stop_words: stopwords.words('english')
 Set to remove all stopwords in English. We do this since they generally don't provide useful discriminative information.

## KNN

In [35]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

In [36]:
one_file.columns

Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date', 'new_review_body',
       'new_review_headline', 'new_product_title', 'rev_dict', 'neg_prop',
       'neu_prop', 'pos_prop', 'compound_prop'],
      dtype='object')

In [37]:
def convert_to_id(x):
    alphabet = {'a': 1, 'c': 3, 'b': 2, 'e': 5, 'd': 4, 'g': 7, 'f': 6, 'i': 9, 'h': 8, 'k': 11, 'j': 10, 'm': 13, 'l': 12, 'o': 15, 'n': 14, 'q': 17, 'p': 16, 's': 19, 'r': 18, 'u': 21, 't': 20, 'w': 23, 'v': 22, 'y': 25, 'x': 24, 'z': 26}
    out = ''
    for i in x:
        if i.lower() in alphabet.keys():
            out += str(alphabet[i.lower()])
        else:
            out += i
    return out

In [38]:
one_file['product_id_convert'] = one_file.get("product_id").apply(convert_to_id)

In [39]:
one_file['product_category_convert'] = one_file.get("product_category").apply(convert_to_id)

In [40]:
one_file['prod_title_comp'] = one_file.get("new_product_title").apply(only_compound)

In [41]:
one_file['rev_title_comp'] = one_file.get("new_review_headline").apply(only_compound)

def help_prop(vals):
    if vals[1] == 0:
        return 0.5
    else:
        return vals[0] / vals[1]

In [83]:
#NEED TO THINK OF WHAT WOULD HAPPEN IF THE NUMBER OF TOTAL VOTES WOULD BE 0
one_file['help_prop'] = one_file.get("helpful_votes") / one_file.get("total_votes")
#one_file['help_prop'] = one_file.get("helpful_votes").apply(help_prop)

In [84]:
one_file.iloc[2].to_frame()

,165832
marketplace,US
customer_id,44099256
review_id,RM8UXQHVQADBP
product_id,B00SMVLUNK
product_parent,950751339
product_title,"Panasonic RP-BTD10-K Premium Bluetooth Wireless On-Ear Headphones, Black"
product_category,Electronics
star_rating,4
helpful_votes,2
total_votes,3


In [44]:
def id_for_dictionary(dic):
    if len(dic) == 4:
        ind = list(dic.values()).index(max(list(dic.values())[0:-1])) #remove the compound
    else:
        ind = list(dic.values()).index(max(list(dic.values())))
        
    if ind == 1:
        return 0
    elif ind == 0:
        return -1
    else:
        return 1

In [45]:
def id_for_prop(prop):
    if prop < 0.45:
        return -1
    elif prop > 0.55:
        return 1
    else:
        return 0

In [46]:
one_file['rev_bod_id'] = one_file.get("rev_dict").apply(id_for_dictionary)

In [47]:
one_file['help_prop_id'] = one_file.get("help_prop").apply(id_for_prop)

find percentages for the body -- text and display those in a column -- act as labels for the percent positivity of the review body and header themselves

In [48]:
#imp_col = one_file[['verified_purchase', 'prod_title_comp', 'star_rating', 'rev_title_comp', 'neg_prop', 'neu_prop', 'pos_prop', 'help_prop']]
imp_col = one_file[['verified_purchase', 'prod_title_comp', 'star_rating', 'rev_title_comp', 'rev_bod_id', 'help_prop_id']]

In [49]:
imp_col.dtypes

verified_purchase     object
prod_title_comp      float64
star_rating            int64
rev_title_comp       float64
rev_bod_id             int64
help_prop_id           int64
dtype: object

In [50]:
#need to replace review_body and review_headline with percentages for sentiment -- easier to use for classification
#vine needs to be converted to a yes/no binary column similar to how positively rated was determined
#use original dataset with rating 3 available

#X = imp_col.iloc[:, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]].values
X = imp_col.iloc[:, [1,2,3,4,5]].values #only taking in the categories that will be used as a dataframe
y = imp_col.iloc[:, 0].values

"""
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,1] = le.fit_transform(X[:,1])
"""

'\nfrom sklearn.preprocessing import LabelEncoder\nle = LabelEncoder()\nX[:,1] = le.fit_transform(X[:,1])\n'

In [51]:
X

array([[ 0.25  ,  5.    ,  0.    ,  1.    ,  0.    ],
       [ 0.    ,  5.    ,  0.6249,  0.    ,  0.    ],
       [ 0.    ,  4.    ,  0.296 ,  0.    ,  1.    ],
       ...,
       [ 0.    ,  1.    ,  0.    ,  0.    ,  1.    ],
       [-0.2732,  5.    ,  0.    ,  1.    ,  0.    ],
       [ 0.    ,  5.    ,  0.6249,  0.    ,  0.    ]])

#Since our dataset containing character variables we have to encode it using LabelEncoder

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [53]:
# Next, we are doing feature scaling to the training and test set of independent variables for reducing the size to smaller values
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [54]:
from sklearn.neighbors import KNeighborsClassifier

#we are using 
#5 neighborhood points are required for classifying a given point -- distance metric is using the minkonowski equation
knn_classifier = KNeighborsClassifier(n_neighbors = 20, metric = 'euclidean', p = 1)
knn_classifier.fit(X_train, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=20, p=1)

In [55]:
y_pred = knn_classifier.predict(X_test)

In [56]:
#We can evaluate our model using the confusion matrix and accuracy score by comparing the predicted and actual test values

from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test, y_pred)
ac = accuracy_score(y_test,y_pred)

In [57]:
print(cm)

[[    7  3779]
 [   24 36187]]


In [58]:
print(ac)

0.904917868840163


In [59]:
#Can see the model performance and add more features accordingly -- 
#would be good if the performance is greater than 85%

### Test on a product review - Need to write a *function* for this for taking in the user input

The features that we are looking at!
* 'prod_title_comp', 
* 'star_rating', 
* 'rev_title_comp', 
* 'rev_bod_id', 
* 'help_prop_id'

In [89]:
review_body = "this is a good review"
product_title = "Sony Headphones"
review_title = 'Love the product!'

star_rating = 5
helpful_votes = 1
total_votes = 1

In [90]:
test = pd.DataFrame()
test['review_body'] = np.array([review_body])
test['review_title'] = np.array([review_title])
test['product_title'] = np.array([product_title])
test

,review_body,review_title,product_title
0,this is a good review,Love the product!,Sony Headphones


In [91]:
out = df_cleaning(test, 'review_body')
out = df_cleaning(out, 'review_title')
out = df_cleaning(out, 'product_title')
out

,review_body,review_title,product_title,new_review_body,new_review_title,new_product_title
0,this is a good review,Love the product!,Sony Headphones,good review,love product,sony headphone


In [95]:
out['review_body'][0]

'this is a good review'

def get_sentiment_proportions(review):
    """
    create new dataframe with just the proportions for each review
    four columns
    neg_prop, pos_prop, neu_prop, compound_prop and will contain these values
    obtained from the vator sentiment algorithm
    """
    snt = analyser.polarity_scores(review)
    #print(f"{sentence} {str(snt)}")
    neg = snt['neg']
    neu = snt['neu']
    pos = snt['pos']
    #compound = snt['compound']
    return neg, neu, pos

neg, neu, pos = get_sentiment_proportions(out.get("new_review_body").iloc[0])

product_category = convert_to_id(product_category)
product_title = only_compound(product_title)
rev_title = only_compound(review_title)

In [100]:
rev_bod_id = id_for_dictionary(analyser.polarity_scores(out['new_review_body'][0]))
help_prop_id = id_for_prop(helpful_votes / total_votes)
prod_title_comp = only_compound(out['new_review_title'][0])
rev_title_comp = only_compound(out['new_product_title'][0])

Predicted: 'verified_purchase'

User Input: 'prod_title_comp', 'product_category_convert', 'star_rating', 'helpful_votes', 'total_votes', 'neg_prop', 'neu_prop', 'pos_prop'
- 8 fields

In [101]:
rev_input_test = np.array([[prod_title_comp, star_rating, rev_title_comp, rev_bod_id, help_prop_id]])
rev_input_test

array([[0.6369, 5.    , 0.    , 1.    , 1.    ]])

In [102]:
prediction, probabilities = knn_classifier.predict(rev_input_test), knn_classifier.predict_proba(rev_input_test)[0]

In [103]:
prediction

array(['Y'], dtype=object)

In [104]:
probabilities

array([0.05, 0.95])

classifier?

In [105]:
def interpret_prediction(review, pred, proba):
    proba = [round(proba[0], 3), round(proba[1], 3)]
    if prediction[0] == 'Y':
        print(f'"{review}" is predicted to be a VERIFIED review, with {proba[1]*100}% probability of being VERIFIED and {proba[0]*100}% probability of being UNVERIFIED')
    if prediction[0] == 'N':
        print(f'"{review}" is predicted to be an UNVERIFIED review, with {proba[0]*100}% probability of being UNVERIFIED and {proba[1]*100}% probability of being VERIFIED')
        
interpret_prediction(review_test, prediction, probabilities)

"this is a good review" is predicted to be a VERIFIED review, with 95.0% probability of being VERIFIED and 5.0% probability of being UNVERIFIED


In [106]:
from joblib import dump, load

In [107]:
knn_classifier

KNeighborsClassifier(metric='euclidean', n_neighbors=20, p=1)

In [108]:
name = 'knn_working_model.joblib'
path = 'KNNModelFiles/'
dump(knn_classifier, path+name)

['KNNModelFiles/knn_working_model.joblib']

In [109]:
knn_classifier = load(path+name)